<a href="https://colab.research.google.com/github/kittipanpip/BADS7201/blob/main/Link_Prediction_in_Inter_orgnatizational_Coordination_Network_of_Flood_Victims_Assitance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install ampligraph

In [ ]:
#!pip uninstall tensorflow
#!pip install tensorflow==1.13.2

In [ ]:
import numpy as np
import pandas as pd
import ampligraph

ampligraph.__version__

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

'1.4.0'

# 1.Dataset exploration


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
src = pd.read_csv("/content/drive/My Drive/data.csv",header=None)
df = src.copy()
X = np.array(df.values.tolist())
print(X.shape)
X = np.unique(X, axis=0)
print(X.shape)
X[:10]

(358, 3)
(225, 3)


array([['Armed Forces Headquarters', 'NEEDS MANAGEMENT', 'Unidentified'],
       ['Department of Disaster Prevention and Mitigation', 'CAPABILITY',
        'Local Authorities'],
       ['Department of Disease Control', 'ASSESSMENT', 'Local Hospital'],
       ['Department of Disease Control', 'ASSESSMENT', 'Unidentified'],
       ['Disaster Prevention and Mitigation Department', 'ASSESSMENT',
        'Royal Thai Navy'],
       ['Disaster Prevention and Mitigation Department', 'ASSET',
        'Unidentified'],
       ['Disaster Prevention and Mitigation Department', 'CAPABILITY',
        'Unidentified'],
       ['Disaster Prevention and Mitigation Department', 'DISSEMINATION',
        'Hydro and Agro Informatics Institute'],
       ['Disaster Prevention and Mitigation Department', 'DISSEMINATION',
        'Ministry of Defense'],
       ['Disaster Prevention and Mitigation Department', 'DISSEMINATION',
        'National Disaster Warning Center']], dtype='<U48')

In [ ]:
entities = np.unique(np.concatenate([X[:, 0], X[:, 2]]))
print(len(entities))
entities[:10]

69


array(['18th Military Circle', 'Agricultural Economic Office',
       'Armed Forces Headquarters', "Ayutthaya's Rama VI dam",
       'Chao Phraya Dam Authority', 'Community Leader',
       'Department of Agricultural Extension',
       'Department of Disaster Prevention and Mitigation',
       'Department of Disease Control',
       'Disaster Prevention and Mitigation Department'], dtype='<U48')

In [ ]:
relations = np.unique(X[:, 1])
print(len(relations))
relations[:10]

22


array(['ASSESSMENT', 'ASSET', 'CAPABILITY', 'CENTER', 'DISSEMINATION',
       'EASE', 'EVACUATION', 'LIMIT', 'MEDICAL', 'NEEDS MANAGEMENT'],
      dtype='<U48')

Import unseen dataset for link prediction

In [ ]:
src2 = pd.read_csv("/content/drive/My Drive/X_unseen.csv",header=None)
df2 = src2.copy()
X_unseen = np.array(df2.values.tolist())

In [ ]:
print(len(X_unseen))

51476


# 2.Defining train and test datasets

In [ ]:
from ampligraph.evaluation import train_test_split_no_unseen 

X_train, X_test = train_test_split_no_unseen(X, test_size=45) 

In [ ]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (180, 3)
Test set size:  (45, 3)


# 3.Train Model

**Ampligraph**

Models: TransE, DistMult, ComplEx

Optimizers: sgd, adagrad, adam, momentum

In [ ]:
from ampligraph.latent_features import TransE
from ampligraph.latent_features import DistMult
from ampligraph.latent_features import ComplEx

In [ ]:
params = {"batches_count": 50, 
                "seed":0, 
                "epochs":200, 
                "k":150, 
                "eta":5,
                "optimizer_params":{'lr':1e-3},
                "loss":'multiclass_nll', 
                "regularizer":'LP', 
                "regularizer_params":{'p':3, 'lambda':1e-5}, 
                "verbose":True
}

models = [
           ('TransE_sgd', TransE(**params, optimizer='sgd')),
           ('TransE_adagrad', TransE(**params, optimizer='adagrad')),
           ('TransE_adam', TransE(**params, optimizer='adam')),
           ('TransE_momentum', TransE(**params, optimizer='momentum')),
           ('DistMult_sgd', DistMult(**params, optimizer='sgd')),
           ('DistMult_adagrad', DistMult(**params, optimizer='adagrad')),
           ('DistMult_adam', DistMult(**params, optimizer='adam')),
           ('DistMult_momentum', DistMult(**params, optimizer='momentum')),
           ('ComplEx_sgd', ComplEx(**params, optimizer='sgd')),
           ('ComplEx_adagrad', ComplEx(**params, optimizer='adagrad')),
           ('ComplEx_adam', ComplEx(**params, optimizer='adam')),
           ('ComplEx_momentum', ComplEx(**params, optimizer='momentum'))   
          ]          

In [ ]:
positives_filter = X

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

for name, model in models:
  print("======== Train Model " + name + "====================")
  model.fit(X_train, early_stopping = False)

======== Train Model TransE_sgd====================


Average TransE Loss:   0.161892: 100%|██████████| 200/200 [00:13<00:00, 15.08epoch/s]


======== Train Model TransE_adagrad====================


Average TransE Loss:   0.634015: 100%|██████████| 200/200 [00:13<00:00, 14.86epoch/s]


======== Train Model TransE_adam====================


...

Average ComplEx Loss:   1.594693: 100%|██████████| 200/200 [00:19<00:00, 10.02epoch/s]


======== Train Model ComplEx_adam====================


Average ComplEx Loss:   0.085600: 100%|██████████| 200/200 [00:20<00:00,  9.54epoch/s]


======== Train Model ComplEx_momentum====================


Average ComplEx Loss:   0.436572: 100%|██████████| 200/200 [00:19<00:00, 10.18epoch/s]


# 4.Evaluating a model

In [ ]:
from ampligraph.evaluation import evaluate_performance

In [ ]:
ranks = []

for name, model in models:
  tmp =  evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)
  ranks.append(("rank_"+ name, tmp))
  tmp = []

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 45/45 [00:00<00:00, 480.73it/s]


WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 45/45 [00:00<00:00, 491.41it/s]


WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


...

100%|██████████| 45/45 [00:00<00:00, 346.71it/s]


WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 45/45 [00:00<00:00, 345.21it/s]


WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 45/45 [00:00<00:00, 350.38it/s]


In [ ]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

for name, rank in ranks:
  print("===============" + name + "====================================")
  print("MRR: %.2f" % mrr_score(rank))
  print("Hits@10: %.2f" % (hits_at_n_score(rank, n=10)))
  print("Hits@3: %.2f" % (hits_at_n_score(rank, n=3)))
  print("Hits@1: %.2f" % (hits_at_n_score(rank, n=1)))


===============rank_TransE_sgd====================================
MRR: 0.34
Hits@10: 0.88
Hits@3: 0.61
Hits@1: 0.04
===============rank_TransE_adagrad====================================
MRR: 0.28
Hits@10: 0.74
Hits@3: 0.47
Hits@1: 0.02
===============rank_TransE_adam====================================
MRR: 0.33
Hits@10: 0.79
Hits@3: 0.49
Hits@1: 0.10
===============rank_TransE_momentum====================================
MRR: 0.30
Hits@10: 0.79
Hits@3: 0.49
Hits@1: 0.04
===============rank_DistMult_sgd====================================
MRR: 0.04
Hits@10: 0.04
Hits@3: 0.00
Hits@1: 0.00
===============rank_DistMult_adagrad====================================
MRR: 0.08
Hits@10: 0.23
Hits@3: 0.04
Hits@1: 0.01
===============rank_DistMult_adam====================================
MRR: 0.29
Hits@10: 0.60
Hits@3: 0.37
Hits@1: 0.11
===============rank_DistMult_momentum====================================
MRR: 0.35
Hits@10: 0.77
Hits@3: 0.42
Hits@1: 0.17
===============rank_ComplEx_sgd=====

# 5.Predicting New Links

In [ ]:
unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))

In [ ]:
ranks_unseen = []

for name, model in models:
  tmp =  evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
  )
  ranks_unseen.append(("rank_unseen_"+ name, tmp))
  tmp = []

100%|██████████| 51476/51476 [01:20<00:00, 642.70it/s]


In [ ]:
scores = []

for name, model in models:
  tmp =  model.predict(X_unseen)
  scores.append(("scores"+ name, tmp))
  tmp = []


In [ ]:
from scipy.special import expit

probs = []

for name, score in scores:
  tmp =  expit(score)
  probs.append(("probs"+ name, tmp))
  tmp = []


In [ ]:
g = []

for index in range(len(models)):
  tmp = pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen[index][1], 
                      np.squeeze(scores[index][1]),
                      np.squeeze(probs[index][1]))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score")
  g.append(("g"+ models[index][0], tmp))
  tmp = []

In [ ]:
for name, g_value in g:
  print("======================="+name+"================")
  print(g_value.sort_values(by=['prob'],ascending=False))

=======================gTransE_sgd================
                                               statement  ...          prob
28574  Local Administration SHELTER Provincial Admini...  ...  1.494845e-06
45333  Office of the Prime Minister SHELTER MANAGEMEN...  ...  7.875685e-07
28561   Local Administration SHELTER Ministry of Defense  ...  5.892174e-07
13627  Disaster Prevention and Mitigation Department ...  ...  5.383280e-07
28564  Local Administration SHELTER Ministry of Interior  ...  4.948232e-07
...                                                  ...  ...           ...
5672   Ayutthaya's Rama VI dam SUPPLIES Industrial Es...  ...  3.668080e-13
5708    Ayutthaya's Rama VI dam SUPPLIES Royal Thai Army  ...  3.196739e-13
45943  Princess Chulabhorn Satellite Receiving Statio...  ...  3.048105e-13
1532   Agricultural Economic Office ASSESSMENT Minist...  ...  2.039364e-13
17710  Federation of Thai Industry ASSESSMENT Ministr...  ...  1.901734e-13

[51476 rows x 4 columns]
===========

In [ ]:
from google.colab import files

for name, g_value in g:
  g_value.to_excel(name+'.xlsx', sheet_name='Result')
  files.download(name+'.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>